In [ ]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [ ]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [ ]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
%pip install -q tabula-py

### **Importing**

In [ ]:
# Criteria for downloading: more than 2 nbsp and href link contains nebula